In [55]:
# Imports
from datetime import datetime as dt
import pandas as pd

In [56]:
# Reading data from source and creating df
storm_df = pd.read_csv('data/events-US-1980-2021.csv',
                       header=1,
                       parse_dates=['Begin Date', 'End Date'])

In [58]:
column_rename = {'Total CPI-Adjusted Cost (Millions of Dollars)': 'cost_in_millions'}
storm_df.rename(columns=column_rename, inplace=True)

In [59]:
storm_df.dtypes

Name                        object
Disaster                    object
Begin Date          datetime64[ns]
End Date            datetime64[ns]
cost_in_millions           float64
Deaths                       int64
dtype: object

In [60]:
storm_df['Disaster'].value_counts()

Disaster
Severe Storm        152
Tropical Cyclone     57
Flooding             36
Drought              29
Winter Storm         20
Wildfire             20
Freeze                9
Name: count, dtype: int64

In [61]:
# Filtering the dataset to match the original visualisation
severe_storm_df = storm_df.loc[storm_df['Disaster'] == 'Severe Storm'].copy(deep=True)
severe_storm_df.reset_index(drop=True, inplace=True)

In [62]:
severe_storm_df

,Name,Disaster,Begin Date,End Date,cost_in_millions,Deaths
0,"Severe Storms, Flash Floods, Hail, Tornadoes (...",Severe Storm,1981-05-05,1981-05-10,1268.6,20
1,Midwest/Plains/Southeast Tornadoes (April 1982),Severe Storm,1982-04-02,1982-04-04,1444.8,33
2,Severe Storms (June 1982),Severe Storm,1982-05-31,1982-06-10,1420.6,30
3,"Tornadoes, Severe Storms, Floods (Spring 1984)",Severe Storm,1984-03-27,1984-04-07,1670.5,80
4,Severe Storms and Hail (June 1984),Severe Storm,1984-06-13,1984-06-17,1222.6,1
...,...,...,...,...,...,...
147,Central Severe Storms (June 2021),Severe Storm,2021-06-24,2021-06-26,1303.0,0
148,Central Severe Storms (July 2021),Severe Storm,2021-07-08,2021-07-11,1123.2,0
149,North Central Severe Weather (August 2021),Severe Storm,2021-08-10,2021-08-13,1335.3,2
150,"Southeast, Central Tornado Outbreak (December ...",Severe Storm,2021-12-10,2021-12-10,3993.3,93


In [63]:
# created columns for day, month and year to assist in future work
# Went with start date as it would be most relevant for investigating, but can change/add to if needed
day_list = [date.day for date in severe_storm_df['Begin Date']]
month_list = [date.month for date in severe_storm_df['Begin Date']]
year_list = [date.year for date in severe_storm_df['Begin Date']]
severe_storm_df['day_start'], severe_storm_df['month_start'], severe_storm_df['year_start'] = day_list, month_list, year_list

In [64]:
# meteorologic seasons are easier to define as they encompass entire months
# can also just do season mapping from the beginning, but like to keep it this way to show different methods and for simplifying steps
met_season_numbers = [month % 12 // 3 + 1 for month in severe_storm_df['month_start']]

season_mapping = {1: 'winter', 2: 'spring', 3: 'summer', 4: 'fall'}

met_season = [season_mapping[number] for number in met_season_numbers]
severe_storm_df['met_season'] = met_season

In [65]:
# Calendar seasons require more logic
# using a start day of the 21st for each season, 3, 6, 9, 12
cal_season = []

for row in severe_storm_df.itertuples():
    if row.month_start in [1,2]:
        cal_season.append('winter')
    elif row.month_start in [4,5]:
        cal_season.append('spring')
    elif row.month_start in [7,8]:
        cal_season.append('summer')
    elif row.month_start in [10,11]:
        cal_season.append('fall')
        
    elif row.month_start == 3:
        if row.day_start < 21:
            cal_season.append('winter')
        else:
            cal_season.append('spring')
    elif row.month_start == 6:
        if row.day_start < 21:
            cal_season.append('spring')
        else:
            cal_season.append('summer')
    elif row.month_start == 9:
        if row.day_start < 21:
            cal_season.append('summer')
        else:
            cal_season.append('fall')
    elif row.month_start == 12:
        if row.day_start < 21:
            cal_season.append('fall')
        else:
            cal_season.append('winter')

severe_storm_df['cal_season'] = cal_season

In [66]:
severe_storm_df

,Name,Disaster,Begin Date,End Date,cost_in_millions,Deaths,day_start,month_start,year_start,met_season,cal_season
0,"Severe Storms, Flash Floods, Hail, Tornadoes (...",Severe Storm,1981-05-05,1981-05-10,1268.6,20,5,5,1981,spring,spring
1,Midwest/Plains/Southeast Tornadoes (April 1982),Severe Storm,1982-04-02,1982-04-04,1444.8,33,2,4,1982,spring,spring
2,Severe Storms (June 1982),Severe Storm,1982-05-31,1982-06-10,1420.6,30,31,5,1982,spring,spring
3,"Tornadoes, Severe Storms, Floods (Spring 1984)",Severe Storm,1984-03-27,1984-04-07,1670.5,80,27,3,1984,spring,spring
4,Severe Storms and Hail (June 1984),Severe Storm,1984-06-13,1984-06-17,1222.6,1,13,6,1984,summer,spring
...,...,...,...,...,...,...,...,...,...,...,...
147,Central Severe Storms (June 2021),Severe Storm,2021-06-24,2021-06-26,1303.0,0,24,6,2021,summer,summer
148,Central Severe Storms (July 2021),Severe Storm,2021-07-08,2021-07-11,1123.2,0,8,7,2021,summer,summer
149,North Central Severe Weather (August 2021),Severe Storm,2021-08-10,2021-08-13,1335.3,2,10,8,2021,summer,summer
150,"Southeast, Central Tornado Outbreak (December ...",Severe Storm,2021-12-10,2021-12-10,3993.3,93,10,12,2021,winter,fall


In [67]:
met_group = severe_storm_df.groupby('met_season')['cost_in_millions'].sum()
print(met_group)

met_season
fall       13503.6
spring    231182.2
summer     72123.2
winter     28114.1
Name: cost_in_millions, dtype: float64


In [68]:
cal_group = severe_storm_df.groupby('cal_season')['cost_in_millions'].sum()
print(cal_group)

cal_season
fall       21082.5
spring    240827.0
summer     44541.5
winter     38472.1
Name: cost_in_millions, dtype: float64


In [54]:
met_group
